<a href="https://colab.research.google.com/github/bernardlawes/Python-PyTorch-DeepLearning/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Convert MNIST Image Files into a Tensor of 4 Dimensions (# of images, Height, Width, Color Channel)

transform = transforms.ToTensor()

In [ ]:
# Train Data - note that datasets is the container that grants us access to MNIST and other datasets available through Torch

train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)

In [ ]:
# Test Data

test_data = datasets.MNIST(root='/cnn_data', train=False, download=True, transform=transform)

In [ ]:
train_data

In [ ]:
test_data

## Step by Step before we Build out Model for Illustrative Purposes

In [ ]:
# Create a small batch size for images ... let's say 10
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [ ]:
# Define our CNN Model - we won't build CNN model here..  we will go about it step by step for this example
# Descrbe Convolutional layer and what it is doing (2 convolutional layers)
# this is just an example to illustrate how it all works

conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1)   # 6 output channels represents 6 filters
conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1)  # 6 outputs will become 6 inputs for this layer

In [ ]:
# Grab 1 MNIST record / image
for i, (X_train, y_train) in enumerate(train_data):
    break

In [ ]:
X_train

In [ ]:
X_train.shape

In [ ]:
# It is curently a 1D image, we want to change it into a 4D torch

X = X_train.view(1,1,28,28)

In [ ]:
X = f.relu(conv1(X)) # rectified lineawr unit for our activation function

In [ ]:
X.shape

In [ ]:
# we can see that we have 1 single image, 6 filters, 26x26 is now the image size (notice the difference is because there is no padding -
# we don't need it for this case) - in real world use cases, we usually will need it

In [ ]:
# pass through our poolling layer
X = f.max_pool2d(X, 2, 2) # kernal size of 2 and stride of 2

In [ ]:
X.shape

In [ ]:
# After poolling, the data is shrunk by 2 because the kernal size and stride is 2 and 2 respectively

In [ ]:
# Let's Do our Second Convolutional Layer

X = f.relu(conv2(X))

In [ ]:
X.shape

In [ ]:
# again, due to no padding, the image shrinks further because we lose 2 pixels around the outside of the image

In [ ]:
#Pooling Layer
X = f.max_pool2d(X, 2, 2)

In [ ]:
X.shape

In [ ]:
# Original image was 28 pixels, lost 2 by conv, lost half by pooling, lost another 2 by conv, and half again by pooling
((28-2)/2-2)/2

# Build out the actual model to go through all images

In [ ]:
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1)
    # Fully Connected Layers
    self.fc1 = nn.Linear(in_features=400, out_features=120) # 5*5*16 - the output fom the extraction
    self.fc2 = nn.Linear(in_features=120, out_features=84)
    self.fc3 = nn.Linear(in_features=84, out_features=10)

  # Note the lowercase X so it doesn't confuse with the capital X used in the step by step section of this colab
  def forward(self, x):
    x = f.relu(self.conv1(x))
    x = f.max_pool2d(x, 2, 2) # 2x2 kernal and stride 2
    # Second Pass
    x = f.relu(self.conv2(x))
    x = f.max_pool2d(x, 2, 2) # 2x2 kernal and stride 2

    # Re-view to flatten it out
    x = x.view(-1, 400)  # negative so that we can vary the batch size

    # Fully Connected Layers
    x = f.relu(self.fc1(x))
    x = f.relu(self.fc2(x))
    x = self.fc3(x)

    return f.log_softmax(x, dim=1)

In [ ]:
# Create an Instance of our Model

seed = 41

torch.manual_seed(seed)
model = ConvolutionalNetwork()
model

In [ ]:
# Loss function optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #smaller the learning rate - the longer it going to train

In [ ]:
import time # so that we can keep track of time

start_time = time.time()

# Create variables to track things
epochs = 5
train_losses = [] # empty python list
test_losses = []
train_correct = []
test_correct = []

# For Loop of Epochs
for i in range(epochs):
  trn_corr = 0 # train correct
  tst_corr = 0 # test correct

  # Train

  for b, (X_train, y_train) in enumerate(train_loader):
    b+=1 # start our batches at 1

    y_pred = model(X_train) # get predcited values from traiing set.  Not flattened, it is 2D
    loss = criterion(y_pred, y_train) # how off are we?  Compare the predictions to the correct answers in y_train

    predicted = torch.max(y_pred.data,1)[1] # add up the number of correct preditions. Indexed off of first point
    batch_corr = (predicted == y_train).sum() # how mahy we got correct from this batch. True = 1. False = 0.
    trn_corr += batch_corr # keep track as we go along in training

    # Update our parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print out our results
    if b%600 == 0:
      print(f'Epoch: {i} | Batch: {b} | Loss: {loss.item()}')

  train_losses.append(loss)
  train_correct.append(trn_corr)

  # Test
  with torch.no_grad(): # No gradient so that we don't update our weights and biases during test iterations
    for b, (X_test, y_test) in enumerate(test_loader): # test_loader instead of train_loader
      y_val = model(X_test) #
      predicted = torch.max(y_val.data, 1)[1] # adding up correct preditions
      tst_corr += (predicted == y_test).sum() # True = 1, False = 0, then sum

  loss = criterion(y_val, y_test)
  test_losses.append(loss)
  test_correct.append(tst_corr)

current_time = time.time()
total_time = current_time - start_time

print(f'Training Tool: {total_time} minutes |')